## Цели исследования

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Задача  — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

Таблица books

Содержит данные о книгах:

•	book_id — идентификатор книги;
•	author_id — идентификатор автора;
•	title — название книги;
•	num_pages — количество страниц;
•	publication_date — дата публикации книги;
•	publisher_id — идентификатор издателя.

Таблица authors

Содержит данные об авторах:

•	author_id — идентификатор автора;
•	author — имя автора.

Таблица publishers

Содержит данные об издательствах:

•	publisher_id — идентификатор издательства;
•	publisher — название издательства;

Таблица ratings

Содержит данные о пользовательских оценках книг:

•	rating_id — идентификатор оценки;
•	book_id — идентификатор книги;
•	username — имя пользователя, оставившего оценку;
•	rating — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах:

•	review_id — идентификатор обзора;
•	book_id — идентификатор книги;
•	username — имя автора обзора;
•	text — текст обзора.


## Исследуем таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Содержание таблиц:

In [3]:
query = """
SELECT *
FROM books
LIMIT 5;
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = """
SELECT *
FROM authors
LIMIT 5;
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = """
SELECT *
FROM publishers
LIMIT 5;
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = """
SELECT *
FROM ratings
LIMIT 5;
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = """
SELECT *
FROM reviews
LIMIT 5;
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи

**Посчитаем, сколько книг вышло после 1 января 2000 года**

In [8]:
query = ('''

SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01';

''') 

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 вышло 819 книг.

**Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [9]:
query = """

SELECT b.title,
       COALESCE(review_cnt, 0) AS review_cnt,
       ROUND(AVG(r.rating), 2) AS rating_avg
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
LEFT JOIN
  (SELECT book_id,
          COUNT(DISTINCT review_id) AS review_cnt
   FROM reviews
   GROUP BY book_id) AS rv ON b.book_id = rv.book_id
GROUP BY b.book_id,
         b.title,
         review_cnt
ORDER BY rating_avg DESC,
         review_cnt DESC;

"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,title,review_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,Moneyball: The Art of Winning an Unfair Game,3,5.00
2,School's Out—Forever (Maximum Ride #2),3,5.00
3,The Ghost Map: The Story of London's Most Terr...,2,5.00
4,Emily of New Moon (Emily #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,Junky,2,2.00
998,His Excellency: George Washington,2,2.00


1000 книг, ранжированных по средней оценке и количеству рецензий. Самая рейтинговая и рецензируемая книга - A Dirty Job (Grim Reaper #1).

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**

In [10]:
query = """

SELECT p.publisher
FROM publishers AS p
INNER JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
HAVING COUNT(b.book_id) =
  (SELECT COUNT(book_id)
   FROM books
   WHERE num_pages > 50
   GROUP BY publisher_id
   ORDER BY COUNT(book_id) DESC
   LIMIT 1);

"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,publisher
0,Penguin Books


Penguin Books - издательство, которое выпускает больше всего книг толщиной более 50 страниц.

**Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**

In [11]:
query = """

SELECT a.author,
       AVG(r.rating) AS mean_rating
FROM authors AS a,
     books AS b,
     ratings AS r
WHERE a.author_id = b.author_id
  AND b.book_id = r.book_id
  AND b.book_id IN
    (SELECT book_id
     FROM ratings
     GROUP BY book_id
     HAVING COUNT(*) >= 50)
GROUP BY a.author
ORDER BY mean_rating DESC
LIMIT 1;

"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling/Mary GrandPré - автор книг с более 50 оценками и с наивысшей средней оценкой 4.287097.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [12]:
query = """

SELECT ROUND(AVG(rereview_count))
FROM
  (SELECT COUNT(review_id) AS rereview_count
   FROM reviews
   WHERE username IN
       (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48)
   GROUP BY username) AS review_counts;
   
"""
ansver = pd.io.sql.read_sql(query, con = engine)
ansver

,round
0,24.0


Пользователи, поставившие более 48 оценок, в среднем пишут по 24 обзора.

## Вывод

В проекте мы совершили несколько запросов SQL b ответили на несколько вопросов с такими резутьтатами:
    
    После 1 января 2000 вышло 819 книг.
    
    1000 книг, ранжированных по средней оценке и количеству рецензий. Самая рейтинговая и рецензируемая книга - A Dirty Job (Grim Reaper #1). 
   
    Penguin Books - издательство, которое выпускает больше всего книг толщиной более 50 страниц.
   
    J.K. Rowling/Mary GrandPré - автор книг с более 50 оценками и с наивысшей средней оценкой 4.287097. 
   
    Пользователи, поставившие более 48 оценок, в среднем пишут по 24 обзора.
    
В целом, проведенный анализ базы данных помогает компании лучше понять предпочтения пользователей, определить наиболее популярные книги и авторов, а также сформулировать ценностное предложение для нового продукта. Эти выводы могут служить основой для принятия решений и планирования дальнейших действий компании.